## Importing the Packages

In [1]:
import pandas as pd
import numpy as np

## Importing the data

In [2]:
raw_csv_data = pd.read_csv('data/Index2018.csv')
df_comp = raw_csv_data.copy()

## Length of the Time Period

In [3]:
df_comp.date.describe()
# Top is not correct

count           6269
unique          6269
top       30/04/2002
freq               1
Name: date, dtype: object

## From Text to Date

In [4]:
df_comp.date = pd.to_datetime(df_comp.date, dayfirst=True)

In [5]:
df_comp.head()

,date,spx,dax,ftse,nikkei
0,1994-01-07,469.90,2224.95,3445.98,18124.01
1,1994-01-10,475.27,2225.00,3440.58,18443.44
2,1994-01-11,474.13,2228.10,3413.77,18485.25
3,1994-01-12,474.17,2182.06,3372.02,18793.88
4,1994-01-13,472.47,2142.37,3360.01,18577.26


In [6]:
df_comp.date.describe()
# Now top is correct

count                    6269
unique                   6269
top       1999-08-23 00:00:00
freq                        1
first     1994-01-07 00:00:00
last      2018-01-29 00:00:00
Name: date, dtype: object

## Setting the Index

In [7]:
# Each value should correspond to a time period.
# We often wish to examine specific chunks of data
# between two concrete dates
df_comp.set_index('date', inplace=True)

In [8]:
df_comp.head()
# df_comp.date.describe() throws an error because
# once "date" becomes an index, we no longer save
# its values as a separate attribute in the data frame

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


## Setting the Desired Frequency

In [9]:
# The values this argument can take are specific
# letter of the alphabet.
# h = hourly
# w = weekly
# d = daily
# m = monthly
# a = annual
# b = business days
df_comp = df_comp.asfreq('b')

In [10]:
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


## Handling Missing Values

In [11]:
df_comp.isna().sum()

spx       8
dax       8
ftse      8
nikkei    8
dtype: int64

In [12]:
# "fillna()" method goes through the entire
# time series and fills values in several
# different ways
# 1) Front filling: Assigns the vlaue of the previous period
# 2) Back filling: Assigns the value for the next period
# 3) Assigning the same value: Assign the average to all the
# missing values within the time series.

In [13]:
df_comp.spx = df_comp.spx.fillna(method='ffill')

In [14]:
df_comp.isna().sum()

spx       0
dax       8
ftse      8
nikkei    8
dtype: int64

In [15]:
df_comp.ftse = df_comp.ftse.fillna(method='bfill')

In [16]:
df_comp.isna().sum()

spx       0
dax       8
ftse      0
nikkei    8
dtype: int64

In [17]:
df_comp.dax = df_comp.dax.fillna(value=df_comp.dax.mean())

In [18]:
df_comp.isna().sum()

spx       0
dax       0
ftse      0
nikkei    8
dtype: int64

In [19]:
df_comp.nikkei = df_comp.nikkei.fillna(value=df_comp.nikkei.mean())

In [20]:
df_comp.isna().sum()

spx       0
dax       0
ftse      0
nikkei    0
dtype: int64

## Simplifying the Dataset

In [21]:
# We will analyze how the S&P 500 performs
# So, we can remove the columns for FTSE, DAX and NIKKEI
# 1) The less data we load, the faster we can manipulate it.
# 2) Clarity. Easier to keep track of the dataset.

# We start by creating a new column, that will be used in the analysis
# That way, we can easily reutilise the code to analyse other markets
df_comp['market_value'] = df_comp.spx

In [22]:
df_comp.describe()

,spx,dax,ftse,nikkei,market_value
count,6277.000000,6277.000000,6277.000000,6277.000000,6277.000000
mean,1288.642547,6080.063363,5423.690398,14597.055700,1288.642547
std,487.868210,2752.604984,1145.568370,4040.545253,487.868210
min,438.920000,1911.700000,2876.600000,7054.980000,438.920000
25%,992.715221,4070.460000,4487.880000,10711.090000,992.715221
50%,1233.761241,5774.380000,5663.300000,15014.040000,1233.761241
75%,1460.250000,7442.660000,6304.630175,17858.420000,1460.250000
max,2872.867839,13559.600000,7778.637689,24124.150000,2872.867839


In [23]:
del df_comp['spx'], df_comp['dax'], df_comp['ftse'], df_comp['nikkei']

In [24]:
df_comp.describe()

,market_value
count,6277.000000
mean,1288.642547
std,487.868210
min,438.920000
25%,992.715221
50%,1233.761241
75%,1460.250000
max,2872.867839


## Splitting the Data

In [25]:
# To conduct successful Machine Learning
# We need to split our data into 2 sets:
# - Training Set
# - Testing Set
# The closer the forecasts, the better the model
# We usually "shuffle" the data before splitting.
# Shuffling is impossible in TS data, because we rely
# upon chronological data
# Which means, the training set should contain data
# from the beginning up to some cut off point.
# While the testing set, should contain all the rest.

# The appropriate size of the training set is debatable
# Too large = performs poorly with new data
# Too small = can't create a model accurate enough
# For this course, instructor will use 80% training data and 20% testing data
size = int(len(df_comp)*0.8)

In [26]:
df = df_comp.iloc[:size]

In [27]:
df_test = df_comp.iloc[size:]

In [28]:
df.tail()

,market_value
date,
2013-04-01,1562.173837
2013-04-02,1570.252238
2013-04-03,1553.686978
2013-04-04,1559.979316
2013-04-05,1553.278930


In [29]:
df_test.head()

,market_value
date,
2013-04-08,1563.071269
2013-04-09,1568.607909
2013-04-10,1587.731827
2013-04-11,1593.369863
2013-04-12,1588.854623
